In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

import asyncio
import nest_asyncio
import aiohttp

In [2]:
history = {
    'authors': [],
    'titles': [],
    'annotations': [],
    'image_urls': [],
    'book_urls': [],
    'genres': []
}

In [3]:
def save_book_data(history, books):
    BASE_URL = 'https://litnet.com'
    
    for book in books:
        author = book.find_all('a', class_='author')
        author = ', '.join([a.get_text() for a in author])
        
        title = book.find('h4', class_='book-title').get_text(strip=True)
        annotation = book.find('span', itemprop='description').get_text(strip=True)
        image_url = book.find('img', itemprop='image')['src']
        book_url = BASE_URL + books[0].find('a', class_='readmore')['href']
        
        history['authors'].append(author)
        history['titles'].append(title)
        history['annotations'].append(annotation)
        history['image_urls'].append(image_url)
        history['book_urls'].append(book_url)
        history['genres'].append('Фэнтези')

In [4]:
FIRST_PAGE = 1
LAST_PAGE = 10

url = 'https://litnet.com/ru/top/fentezi?alias=fentezi&page={}'
urls = [url.format(i) for i in range(FIRST_PAGE,LAST_PAGE)]

In [5]:
async def fetch(session, url):
    async with session.get(url) as response:
        html = await response.text()
        soup = BeautifulSoup(html, 'html.parser')
        books = soup.find_all('div', class_='row book-item')
        save_book_data(history, books)

In [6]:
async def main():
    async with aiohttp.ClientSession() as session:
        tasks = [fetch(session, url) for url in urls]
        await asyncio.gather(*tasks)

In [7]:
nest_asyncio.apply()
await main()

In [8]:
books_df = pd.DataFrame({
    'author': history['authors'],
    'title': history['titles'],
    'annotation': history['annotations'],
    'img_url': history['image_urls'],
    'book_url': history['book_urls'],
    'genre': history['genres']
})

In [9]:
books_df

,author,title,annotation,img_url,book_url,genre
0,Ольга Сурмина,Не Альфа волчьей стаи,Она всю жизнь готовилась выйти замуж за Альфу....,https://rust.litnet.com/uploads/covers/120/174...,https://litnet.com/ru/book/ne-alfa-volchei-sta...,Фэнтези
1,"Юлия Арниева, Ульяна Заповедная",Несносные дочери леди Элизабет 2,"Я вышла замуж по договорённости — без чувств, ...",https://rust.litnet.com/uploads/covers/120/174...,https://litnet.com/ru/book/ne-alfa-volchei-sta...,Фэнтези
2,Салма Кальк,Виктория - значит Победа. Сердцу не прикажешь,"Я победила вздорных родственников, алчных сосе...",https://rust.litnet.com/uploads/covers/120/173...,https://litnet.com/ru/book/ne-alfa-volchei-sta...,Фэнтези
3,Дарья Верескова,Личный маг для Наследника. Эхо погибшей цивили...,— Я заплачу тебе. Сколько ты хочешь? Десять се...,https://rust.litnet.com/uploads/covers/120/173...,https://litnet.com/ru/book/ne-alfa-volchei-sta...,Фэнтези
4,Эми Эванс,( Не)нужная жена дракона. Хозяйка вишневых садов,НОВИНКА: https://litnet.com/shrt/9P9d\n\nЯ поп...,https://rust.litnet.com/uploads/covers/120/174...,https://litnet.com/ru/book/ne-alfa-volchei-sta...,Фэнтези
...,...,...,...,...,...,...
175,Марьяна Брай,Страшилище,"Если тебя называют страшилищем, потому что ты ...",https://rust.litnet.com/uploads/covers/120/174...,https://litnet.com/ru/book/nepokornaya-istinna...,Фэнтези
176,Карина Демина,Ведьмы.Ру 2,Все те же плюс демон.\n\nСпасаем друга. Решаем...,https://rust.litnet.com/uploads/covers/120/174...,https://litnet.com/ru/book/nepokornaya-istinna...,Фэнтези
177,"Джейд Дэвлин, Рина Ских",Седьмая жизнь злодейки,"Я была доброй и глупой, но меня выставили злод...",https://rust.litnet.com/uploads/covers/120/174...,https://litnet.com/ru/book/nepokornaya-istinna...,Фэнтези
178,Лилиана Хоффман,"Действуй, Ведьма! или игра судьбы","🔥 Книга публикуется бесплатно. Черновик! Да, ...",https://rust.litnet.com/uploads/covers/120/172...,https://litnet.com/ru/book/nepokornaya-istinna...,Фэнтези


In [10]:
books_df.to_csv('books.csv', index=False)